<a href="https://colab.research.google.com/github/elichen/aoc2017/blob/main/Day_14_Disk_Defragmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
def knot2(lengths, numbers, current_position, skip_size):
    for length in lengths:
        sublist = [numbers[(current_position + i) % len(numbers)] for i in range(length)]
        sublist.reverse()
        for i in range(length):
            numbers[(current_position + i) % len(numbers)] = sublist[i]
        current_position = (current_position + length + skip_size) % len(numbers)
        skip_size += 1

    return numbers, current_position, skip_size

from functools import reduce
from operator import xor
def dense_hash(l):
    return [reduce(xor,x) for x in [l[i:i+16] for i in range(0,256,16)]]

def knot_hash(s):
  lengths = [ord(c) for c in s] + [17, 31, 73, 47, 23]
  numbers = list(range(256))
  current_position = 0
  skip_size = 0
  for _ in range(64):
    numbers, current_position, skip_size = knot2(lengths, numbers, current_position, skip_size)
  dense = dense_hash(numbers)
  return("".join([f'{x:02x}' for x in dense]))

In [24]:
data = "flqrgnkx"
data = "ffayrhll"

In [25]:
total = 0
for i in range(128):
  hex_string = knot_hash(f"{data}-{i}")
  binary_string = bin(int(hex_string, 16))[2:]
  binary_string = binary_string.zfill(len(hex_string) * 4)
  bit_count = binary_string.count('1')
  total += bit_count
total

8190

In [26]:
import numpy as np
from collections import deque

def find_regions(data):
    grid_size = 128
    grid = np.zeros((grid_size, grid_size), dtype=int)

    # Convert hex strings to binary and fill the numpy array
    for i in range(grid_size):
        hex_string = knot_hash(f"{data}-{i}")
        binary_string = bin(int(hex_string, 16))[2:].zfill(len(hex_string) * 4)
        for j in range(grid_size):
            grid[i, j] = int(binary_string[j])

    visited_sets = []

    # Function to perform BFS
    def bfs(start):
        visited = set()
        queue = deque([start])
        while queue:
            x, y = queue.popleft()
            if (x, y) not in visited:
                visited.add((x, y))
                for dx, dy in [(1,0), (-1,0), (0,1), (0,-1)]: # Adjacent squares
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < grid_size and 0 <= ny < grid_size and grid[nx, ny] == 1 and (nx, ny) not in visited:
                        queue.append((nx, ny))
        return visited

    # Iterate over each bit in the grid
    for i in range(grid_size):
        for j in range(grid_size):
            if grid[i, j] == 1 and not any((i, j) in visited_set for visited_set in visited_sets):
                new_visited_set = bfs((i, j))
                visited_sets.append(new_visited_set)

    return visited_sets

visited_sets = find_regions(data)

len(visited_sets)

1134